# Relaxation Problem 2 for Branch Cut

Given integer $N$ and array $P$ of length $2^N$.

We want to evaluate the answer of the following problem:
$$
    \max_{Q \in \mathbb{F}_2^{n \times n}, c \in \mathbb{F}_2^{n}} (-1)^{x^\top Q x} i^{c^\top x} P_x
$$

## naive method

In [12]:
import math

import numpy as np
from numba import njit



@njit(cache=True)
def xQc_to_coeff(k, x, Q, c):
    cx = 0
    xQx = 0

    Q_idx = 0

    for i in range(k):

        cx ^= ((x >> i) & 1) * ((c >> i) & 1)

        for j in range(i, k):

            if ((Q >> Q_idx) & 1) & ((x >> i) & 1) & ((x >> j) & 1):

                xQx ^= 1

            Q_idx += 1

    return (-1) ** xQx * (complex(0, 1) ** cx)



@njit(cache=True)
def calc_abs_from_bQc(k, b, Q, c):

    ret = complex(0.0, 0.0)

    for x in range(1 << k):

        ret += xQc_to_coeff(k, x, Q, c) * b[x]

    return abs(ret)



def solveSlow(N: int, Xs: list):

    assert len(Xs) == 1 << N

    k = N

    maxAbs = 0.0

    for Q in range(1 << (k * (k + 1) // 2)):

        for c in range(1 << k):

            absVal = calc_abs_from_bQc(k, Xs, Q, c)

            maxAbs = max(maxAbs, absVal)

    return maxAbs

## upper bound 1

In [7]:
def threshold1(N: int, Xs: list):
    return sum(np.abs(Xs))

## upper bound 2

Please refer to `relaxation_problem_1_for_branch_cut.ipynb` for detail.

In [8]:
def rotate(x):
    if x.imag >= 0:
        if x.real >= 0:
            return x
        else:
            return x * -1j
    else:
        if x.real <= 0:
            return -x
        else:
            return x * 1j


def threshold2(N: int, Xs: list):
    assert len(Xs) == 1 << N
    rotated_Xs = [rotate(X) for X in Xs]
    Ys = rotated_Xs.copy()
    Ys.sort(key=lambda Y: (Y.imag / Y.real) if Y.real != 0 else 1e9)
    sumYsReal = sum(Y.real for Y in Ys)
    sumYsImag = sum(Y.imag for Y in Ys)
    maxAbs2 = sumYsReal**2 + sumYsImag**2
    for i in range((1 << N) - 1):
        sumYsReal += -Ys[i].real - Ys[i].imag
        sumYsImag += -Ys[i].imag + Ys[i].real
        absVal2 = sumYsReal**2 + sumYsImag**2
        maxAbs2 = max(maxAbs2, absVal2)
        Ys[i] *= 1j
    return maxAbs2**0.5

## check performance

In [22]:
import random

CNT = 100
N = 4
ratio2 = 0.0
ratioS = 0.0
for seed in range(CNT):
    random.seed(seed)
    Xs = []
    for x in range(1 << N):
        absolute = random.normalvariate(0, 1)
        phase = random.uniform(0, 2 * math.pi)
        Xs.append(absolute * (math.cos(phase) + math.sin(phase) * 1j))
    slow = solveSlow(N, Xs)
    print(f"{slow=}", end=" ")
    t1 = threshold1(N, Xs)
    print(f"{t1=}", end=" ")
    t2 = threshold2(N, Xs)
    print(f"{t2=}")
    ratio2 += t2 / t1
    ratioS += slow / t1

print(f"{ratio2 / CNT=}")
print(f"{ratioS / CNT=}")

slow=9.351250940736668 t1=10.755989621695186 t2=10.22082929238501
slow=9.926630193976946 t1=11.882465145085684 t2=10.88870965974689
slow=15.765857115752345 t1=19.75152320564026 t2=18.282898873327806
slow=8.265982092489573 t1=9.74483980675003 t2=8.953891468319942
slow=9.335748087123404 t1=10.639158229647752 t2=10.07508145329835
slow=8.403200802446896 t1=10.684957073365915 t2=9.835518306946595
slow=12.910648359332466 t1=16.01261363094745 t2=14.6001717471925
slow=12.051254197008696 t1=14.699144686154364 t2=13.79622026000481
slow=12.863664512283814 t1=16.314307869697267 t2=15.049187045564752
slow=11.57333698298256 t1=13.743281948577108 t2=13.233752593524123
slow=9.881288468288798 t1=11.600964070901702 t2=11.039076883908637
slow=9.829977444728996 t1=11.75976564044735 t2=11.18673276575851
slow=8.638114054716247 t1=11.248130118710527 t2=10.584855458638117
slow=10.604790763299885 t1=12.866172101534385 t2=12.15533279393732
slow=7.252358148205841 t1=8.746359974884578 t2=7.994502248723072
slow=9.